1 给Qwen1.5添加\<image\> token

In [ ]:
from transformers import Qwen2TokenizerFast, AutoModelForCausalLM

In [ ]:
llm_path = "model/qwen1.5_0.5b"
llm_tokenizer = Qwen2TokenizerFast.from_pretrained(llm_path, device_map="cuda:0", local_files_only=True)
llm_tokenizer.encode("<image>")

In [ ]:
qwen_model = AutoModelForCausalLM.from_pretrained(llm_path, device_map="cuda:0")
print(qwen_model.model)
print(qwen_model.model.embed_tokens)

2 初始化Model

In [1]:
from transformers import AutoProcessor, AutoTokenizer, AutoModel, AutoModelForCausalLM

clip_path, llm_path = "model/clip" ,"model/qwen1.5_0.5b"
clip_processor = AutoProcessor.from_pretrained(clip_path, device_map="cuda:0")
llm_tokenizer = AutoTokenizer.from_pretrained(llm_path, device_map="cuda:0")
clip_model = AutoModel.from_pretrained(clip_path, device_map="cuda:0")
llm_model = AutoModelForCausalLM.from_pretrained(llm_path, device_map="cuda:0")
clip_config = clip_model.vision_model.config
llm_config = llm_model.config

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
D:\ProgramData\Anaconda3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
clip_model

In [ ]:
llm_model

In [2]:
from transformers import LlavaProcessor, LlavaForConditionalGeneration, LlavaConfig
llava_config = LlavaConfig(vision_config=clip_config, text_config=llm_config)
llava_model = LlavaForConditionalGeneration(llava_config)

In [ ]:
llava_model

3 替换llava model的视觉和文本模型

In [3]:
llava_model.vision_tower.vision_model = clip_model.vision_model
llava_model.language_model = llm_model

4 替换pad_token_id和image_token_index

In [4]:
llava_model.config.pad_token_id = llm_tokenizer.pad_token_id
llava_model.config.image_token_index = llm_tokenizer.encode("<image>")[0]

5 保存模型

In [5]:
llava_path = "model/llava"
llava_model.save_pretrained(llava_path)
llm_tokenizer.save_pretrained(llava_path)

('model/llava\\tokenizer_config.json',
 'model/llava\\special_tokens_map.json',
 'model/llava\\vocab.json',
 'model/llava\\merges.txt',
 'model/llava\\added_tokens.json',
 'model/llava\\tokenizer.json')

6 添加clip的processor到llava模型中
将clip的preprocessor_config.json复制粘贴到llava文件夹下

In [ ]:
clip_processor.save_pretrained("model/processor")

7 尝试体验和加载模型

In [3]:
from PIL import Image
from transformers import LlavaProcessor, LlavaForConditionalGeneration

url = "images/001.jpg"
image = Image.open(fp=url)

prompt_text = "<image>\nWhat are there?"
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": prompt_text}
]

llava_path = "model/llava"
llava_processor = LlavaProcessor.from_pretrained(llava_path, device_map="cuda:0")
prompt = llava_processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_propmt=True)
inputs = llava_processor(text=prompt, images=image)
llava_model = LlavaForConditionalGeneration.from_pretrained(llava_path, device_map="cuda:0")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


array([[[-0.15723877, -0.12804192, -0.11344349, ..., -0.15723877,
          0.8354542 ,  1.2004149 ],
        [-0.18643562, -0.12804192, -0.15723877, ...,  0.32450938,
          1.1274228 ,  1.1858164 ],
        [-0.18643562, -0.08424664, -0.06964822, ...,  0.7332653 ,
          1.1566195 ,  1.1274228 ],
        ...,
        [ 0.03254076,  0.00334391, -0.02585294, ...,  0.0617376 ,
          0.14932826,  0.20772196],
        [ 0.0617376 ,  0.00334391, -0.0550498 , ...,  0.12013142,
          0.19312355,  0.23691882],
        [ 0.0617376 ,  0.00334391, -0.06964822, ...,  0.17852512,
          0.2223204 ,  0.16392669]],

       [[-0.08623484, -0.05621931, -0.04121154, ..., -0.2813358 ,
          0.7391925 ,  1.1894255 ],
        [-0.11625037, -0.05621931, -0.08623484, ...,  0.21392061,
          1.0693634 ,  1.1744177 ],
        [-0.11625037, -0.011196  ,  0.00381176, ...,  0.66415364,
          1.1293944 ,  1.1444021 ],
        ...,
        [ 0.06384283,  0.0338273 ,  0.0338273 , ...,  

In [5]:
for tk in inputs.keys():
    inputs[tk] = inputs[tk].to(llava_model.device)
generate_ids = llava_model.generate(**inputs, max_new_tokens=20)
gen_text = llava_processor.batch_decode(
    generate_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False
)[0]

print(gen_text)

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
<image>
What are there?<|im_end|>
<|im_start|>There are many different types of "news" that can be written in the English language. Some


In [1]:
from transformers import LlavaProcessor, LlavaForConditionalGeneration

llava_path = "model/llava"
llava_model = LlavaForConditionalGeneration.from_pretrained(llava_path, device_map="cuda:0")
# llava_model.vision_tower.vision_model.requires_grad_=False
# llava_model.language_model.requires_grad_=False
# llava_model.multi_modal_projector.requires_grad_ = True


AttributeError: 'LlavaForConditionalGeneration' object has no attribute 'requires_grad'

In [4]:
llava_model

LlavaForConditionalGeneration(
  (vision_tower): CLIPVisionModel(
    (vision_model): CLIPVisionTransformer(
      (embeddings): CLIPVisionEmbeddings(
        (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
        (position_embedding): Embedding(577, 1024)
      )
      (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (encoder): CLIPEncoder(
        (layers): ModuleList(
          (0-23): 24 x CLIPEncoderLayer(
            (self_attn): CLIPAttention(
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
            (mlp): CLIPMLP(
              (activation_fn): Quick

In [6]:
for name, parameters in llava_model.named_parameters():
    if parameters.requires_grad:
        print(name)

vision_tower.vision_model.embeddings.class_embedding
vision_tower.vision_model.embeddings.patch_embedding.weight
vision_tower.vision_model.embeddings.position_embedding.weight
vision_tower.vision_model.pre_layrnorm.weight
vision_tower.vision_model.pre_layrnorm.bias
vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.weight
vision_tower.vision_model.encoder.layers.0.self_attn.k_proj.bias
vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.weight
vision_tower.vision_model.encoder.layers.0.self_attn.v_proj.bias
vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.weight
vision_tower.vision_model.encoder.layers.0.self_attn.q_proj.bias
vision_tower.vision_model.encoder.layers.0.self_attn.out_proj.weight
vision_tower.vision_model.encoder.layers.0.self_attn.out_proj.bias
vision_tower.vision_model.encoder.layers.0.layer_norm1.weight
vision_tower.vision_model.encoder.layers.0.layer_norm1.bias
vision_tower.vision_model.encoder.layers.0.mlp.fc1.weight
vision_tower.vision_mo

In [12]:
llava_model.config.use_cache=False
llava_model.config

LlavaConfig {
  "_name_or_path": "model/llava",
  "architectures": [
    "LlavaForConditionalGeneration"
  ],
  "ignore_index": -100,
  "image_token_index": 151646,
  "model_type": "llava",
  "pad_token_id": 151643,
  "projector_hidden_act": "gelu",
  "text_config": {
    "_name_or_path": "model/qwen1.5_0.5b",
    "architectures": [
      "Qwen2ForCausalLM"
    ],
    "bos_token_id": 151643,
    "eos_token_id": 151645,
    "hidden_size": 1024,
    "intermediate_size": 2816,
    "max_position_embeddings": 32768,
    "max_window_layers": 21,
    "model_type": "qwen2",
    "num_attention_heads": 16,
    "num_hidden_layers": 24,
    "num_key_value_heads": 16,
    "rope_theta": 1000000.0,
    "sliding_window": 32768,
    "tie_word_embeddings": true,
    "torch_dtype": "bfloat16",
    "use_sliding_window": false,
    "vocab_size": 151936
  },
  "torch_dtype": "float32",
  "transformers_version": "4.42.4",
  "use_cache": false,
  "vision_config": {
    "dropout": 0.0,
    "hidden_size": 1024,